# BERT Practical

Practical work on BERT for the course Natural Language Processing in M2 MoSIG

> Author: Archit YADAV

## 1. IMPORT MODULES

In [2]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 
     |████████████████████████████████| 895 kB 68.0 MB/s 
     |████████████████████████████████| 61 kB 478 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
import transformers

# Managing arrays
import numpy as np

In [4]:
# load the TensorBoard notebook extension
# %load_ext tensorboard

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

Will work on CPU.


## 2. DATA

### 2.1 Downloading of Data

In [11]:
from sklearn.datasets import fetch_20newsgroups

categories = [
 'comp.windows.x',
 'sci.med',
 'soc.religion.christian',
 'talk.politics.guns',
]

# Download data if not already present in data_home
trainset = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, data_home='./scikit_learn_data')
testset = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, data_home='./scikit_learn_data')

# Define input data and labels for training and testing
x_train = trainset.data
y_train = trainset.target
x_test = testset.data
y_test = testset.target

# SOLUTION (yes, we are cool)
print('Dataset size: \n{} posts in total'.format(len(x_train) + len(x_test)))
print('\t {} training posts'.format(len(x_train)))

for i in range(len(categories)):
  num = sum(y_train == i)
  print("\t\t {} {}".format(num, categories[i]))

print('\t {} testing posts'.format(len(x_test)))
for i in range(len(categories)):
  num = sum(y_test == i)
  print("\t\t {} {}".format(num, categories[i]))



# print('\n')
# print('EXAMPLE: \n')
# print(x_train[0])



Dataset size: 
3885 posts in total
	 2332 training posts
		 593 comp.windows.x
		 594 sci.med
		 599 soc.religion.christian
		 546 talk.politics.guns
	 1553 testing posts
		 395 comp.windows.x
		 396 sci.med
		 398 soc.religion.christian
		 364 talk.politics.guns


### 2.2 Cleaning of Data

In [12]:
# Remove lines starting with certain keywords 
def clean_post(post: str, remove_start: tuple):
    clean_lines = []
    for line in post.splitlines():
            if not line.startswith(remove_start):
                clean_lines.append(line)
    return '\n'.join(clean_lines)
    

# SOLUTION (yes, again, we are cool)
remove_start = (
  'From:',
  'Subject:',
  'Reply-To:',
  'In-Reply-To:',
  'Nntp-Posting-Host:',
  'Organization:',
  'X-Mailer:',
  'In article <',
  'Lines:',
  'NNTP-Posting-Host:',
  'Summary:',
  'Article-I.D.:'
)
x_train = [clean_post(p, remove_start) for p in x_train]
x_test = [clean_post(p, remove_start) for p in x_test]


## 3. TOKENISATION

In [13]:
from transformers import DistilBertTokenizer

MAX_LEN = 512

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', padding=True, truncation=True)

# Let's check out how the tokenizer works
for n in range(3):
    # Tokenize forum post
    tokenizer_out = tokenizer(x_train[n])
    # Convert numerical tokens to alphabetical tokens
    encoded_tok = tokenizer.convert_ids_to_tokens(tokenizer_out.input_ids)
    # Decode tokens back to string
    decoded = tokenizer.decode(tokenizer_out.input_ids)
    print(tokenizer_out)
    print(encoded_tok, '\n')
    print(decoded, '\n')
    print('---------------- \n')



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (744 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [101, 1996, 13758, 1006, 10900, 1015, 1024, 2570, 1007, 1996, 13758, 2758, 1010, 1036, 1036, 2054, 2003, 3606, 1029, 1005, 1005, 1998, 4332, 2000, 19021, 2243, 1999, 1996, 17005, 1997, 2010, 12746, 1012, 2129, 2715, 2129, 15236, 2129, 5622, 5677, 5844, 2129, 2715, 2129, 15236, 2129, 5622, 5677, 5844, 2027, 16883, 1996, 13758, 1010, 2108, 2137, 2987, 1005, 1056, 2113, 2008, 2010, 1036, 1036, 3160, 1005, 1005, 2715, 18386, 26881, 2001, 2356, 2077, 1010, 2011, 1037, 2715, 18386, 26881, 3923, 2063, 24686, 3761, 1006, 2805, 2025, 2700, 1007, 2040, 3047, 2000, 2444, 2048, 4595, 2086, 3283, 1012, 2066, 2116, 8801, 2002, 8725, 2625, 2055, 15084, 2084, 3463, 2625, 2055, 4515, 2084, 2965, 2625, 2055, 2505, 2084, 4363, 2010, 3105, 1006, 1998, 2010, 2132, 1007, 1012, 2057, 2453, 2655, 2032, 1037, 2978, 12077, 2295, 1036, 3813, 1005, 2052, 2022, 2785, 2121, 1006, 1998, 2053, 4797, 13125, 1010, 2040, 2292, 6343, 2175, 1010, 4191, 2012, 2010, 27327, 2791, 1007, 2002, 2134, 1005, 1056, 2

In [14]:
from torch.utils.data import Dataset, DataLoader

MAX_LEN = 512

class PostsDataset(Dataset):
    def __init__(self, posts, labels, tokenizer, max_len):
        # Variables that are set when the class is instantiated
        self.posts = posts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.posts)
  
    def __getitem__(self, item):
        # Select the post and its category
        post = str(self.posts[item])
        label = self.labels[item]
        # Tokenize the post
        tokenizer_out = self.tokenizer(
            post,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
            )
        # Return a dictionary with the output of the tokenizer and the label
        return  {
            'input_ids': tokenizer_out['input_ids'].flatten(),
            'attention_mask': tokenizer_out['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


# Instantiate two PostsDatasets
train_dataset = PostsDataset(x_train, y_train, tokenizer, MAX_LEN)
test_dataset = PostsDataset(x_test, y_test, tokenizer, MAX_LEN)

## 4. MODEL

In [15]:

from transformers import DistilBertModel

PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
distilbert = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
 
first_post = train_dataset[0]

hidden_state = distilbert(
    input_ids=first_post['input_ids'].unsqueeze(0),
    attention_mask=first_post['attention_mask'].unsqueeze(0)
    )

print(hidden_state[0].shape)
print(distilbert.config)



Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the b

torch.Size([1, 512, 768])
DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.14.1",
  "vocab_size": 30522
}



In [18]:
from transformers import DistilBertPreTrainedModel, DistilBertConfig


PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'

class DistilBertForPostClassification(DistilBertPreTrainedModel):
    def __init__(self, config, num_labels, freeze_encoder=False):
        # Instantiate the parent class DistilBertPreTrainedModel
        super().__init__(config)
        # Instantiate num. of classes
        self.num_labels = num_labels
        # Instantiate and load a pretrained DistilBERT model as encoder
        self.encoder = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        # Freeze the encoder parameters if required (Q1)
        if freeze_encoder:
          for param in self.encoder.parameters():
              param.requires_grad = False
        # The classifier: a feed-forward layer attached to the encoder's head
        self.classifier = torch.nn.Linear(
            in_features=config.dim, out_features=self.num_labels, bias=True)
        # Instantiate a dropout function for the classifier's input
        self.dropout = torch.nn.Dropout(p=0.1)


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        # Encode a batch of sequences with DistilBERT
        encoder_output = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        # Extract the hidden representations from the encoder output
        hidden_state = encoder_output[0]  # (bs, seq_len, dim)
        # Only select the encoding corresponding to the first token
        # of each sequence in the batch (Q2)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        # Apply dropout
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        # Feed into the classifier
        logits = self.classifier(pooled_output)  # (bs, dim)

        outputs = (logits,) + encoder_output[1:]
        
        if labels is not None: # (Q3)
          # Instantiate loss function
          # SOLUTION :
          loss_fct = torch.nn.CrossEntropyLoss()
          # Calculate loss
          # SOLUTION :
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
          # Aggregate outputs
          outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


# Instantiate model
model = DistilBertForPostClassification(
    config=distilbert.config, num_labels=len(categories), freeze_encoder = True
    )

# Print info about model's parameters
total_params = sum(p.numel() for p in model.parameters())
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
trainable_params = sum([np.prod(p.size()) for p in model_parameters])
print('Model total params: ', total_params)
print('Model trainable params: ', trainable_params)
print('\n', model)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.14.1",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9

Model total params:  66365956
Model trainable params:  3076

 DistilBertForPostClassification(
  (encoder): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
     

## 5. TRAINING

In [ ]:

from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',          
    logging_dir='./logs',
    logging_first_step=True,
    logging_steps=50,
    num_train_epochs=4,              
    per_device_train_batch_size=8,  
    learning_rate=5e-5,
    weight_decay=0.01        
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    compute_metrics=compute_metrics
)

train_results = trainer.train()

test_results = trainer.predict(test_dataset=test_dataset)

print('Predictions: \n', test_results.predictions)
print('\nAccuracy: ', test_results.metrics['eval_accuracy'])
print('Precision: ', test_results.metrics['eval_precision'])
print('Recall: ', test_results.metrics['eval_recall'])
print('F1: ', test_results.metrics['eval_f1'])
print(categories)

MODEL_PATH = './my_model'
trainer.save_model(MODEL_PATH)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 2332
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1168
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to th

Step,Training Loss
1,1.510100
50,1.409900
100,1.370900
150,1.339700
200,1.310000
250,1.287400
300,1.244300
350,1.241100
400,1.193400
450,1.200600


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## 6. PREDICTIONS

In [ ]:

model = DistilBertForPostClassification.from_pretrained(
    './my_model', config=distilbert.config, num_labels=len(categories)).to(device)
for sentence in ['Lung cancer is a deadly disease.', 'God is love', 'How can you install Microsoft Office extensions?', 'Gun killings increase every year.']:
  encoding = tokenizer.encode_plus(sentence)
  encoding['input_ids'] = torch.tensor([encoding.input_ids]).to(device)
  encoding['attention_mask'] = torch.tensor(encoding.attention_mask).to(device)
  out = model(**encoding)
  categories_probability = torch.nn.functional.softmax(out[0], dim=1).flatten()
  print(sentence)
  print('\tProbabilities assigned by the model : ')
  for n,c in enumerate(categories):
    print('\t\t{} : {}'.format(c, categories_probability[n]))
  print('\n\t--> Prediction :', categories[categories_probability.argmax()])
  print('------------------------------------------------\n')
  

## 7. IMPROVE THE MODEL

In [ ]:

# # SOLUTION 1 (trivial): increase training epochs
# # SOLUTION 2: finetune encoder parameters too

# model_unfreezed = DistilBertForPostClassification(config, freeze_decoder = False)
# trainer_unfreezed = Trainer(
#     model=model_unfreezed,                         
#     args=training_args,                  
#     train_dataset=train_dataset,         
#     compute_metrics=compute_metrics
# )
# trainer_unfreezed.train()
# trainer_unfreezed.predict(test_dataset=test_dataset)

# # SOLUTION 3: let's see what students can do !
